#shift+enter 从这里开始

In [12]:
import import_ipynb
import oss2
from setting import SETTINGS
import pickle
import pandas as pd
import os
import io
from dateutil import parser
import datetime
import pandas as pd
from fastparquet import write
from pathlib import  Path
from oss_handler import OssClient
import sys, os
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from pylab import mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime
from datetime import datetime, timedelta
from oss2.exceptions import NoSuchKey
import warnings

warnings.filterwarnings('ignore')

AccessKeyId = SETTINGS["oss.accesskey"]
AccessKeySecret = SETTINGS["oss.secret"]
BucketName = SETTINGS["oss.bucketname"]
Endpoint = SETTINGS["oss.endpoint"]


class newBytes(io.BytesIO):
    def close(self):
        pass


class OssClient(object):
    __instance = None
    __first_init = False

    # 单例模式
    def __new__(cls, *args, **kwargs):
        if not cls.__instance:
            cls.__instance = super().__new__(cls)
        return cls.__instance

    def __init__(self):
        cls = self.__class__
        if not cls.__first_init:
            self.auth = oss2.Auth(AccessKeyId, AccessKeySecret)
            self.bucket = oss2.Bucket(self.auth, Endpoint, BucketName)
            cls.__first_init = True


    def upload_file_from_fileobj(self, object_name, local_file_path):
        """
            upload_file_from_fileobj方法：上传文件对象到oss存储空间, 该方法可用于我们从上游服务接收了图片参数，然后以二进制形式读文件，上传到oss存储空间指定位置（abc/efg/00），
        当然也可以将本地文件上传到oss我们的bucket. 其中fileobj不止可以是文件对象，也可以是本地文件路径。 put_object方法底层仍是RESTful API的调用，可以指定headers，规定Content-Type等内容
        """
        # 判断bucket中文件是否存在，也可以不判断，会上传更新
        #exist = self.bucket.object_exists(object_name) #<yourObjectName>
        #if exist:
        #    return True
        with open(local_file_path, 'rb') as fileobj:
            result = self.bucket.put_object(object_name, fileobj) #<yourObjectName>
        if result.status == 200:
            return True
        else:
            return False

    def upload_pickle_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        pickle_buffer = io.BytesIO()
        pickle.dump(df, pickle_buffer)
        target_file_key = os.path.join(target_path, '{}.pkl'.format(d)).replace("\\","/")
        result = self.bucket.put_object(target_file_key, pickle_buffer.getvalue())
        if result.status == 200:
            return True
        else:
            return False


    def list_files(self,prefix = None):
        res = []
        for object_info in oss2.ObjectIterator(self.bucket,prefix):
            print(object_info.key)
            res.append(object_info.key)
        return res

    def upload_parquet_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(target_path, '{}.parquet'.format(d)).replace("\\","/")
        mem_buffer = newBytes()
        df.to_parquet('noname', engine='fastparquet', open_with=lambda x, y: mem_buffer)
        result = self.bucket.put_object(target_file_key, mem_buffer.getvalue())
        #f = Path(os.getcwd())/'tmp.parquet'
        #write(f, df)
        #with open(f, 'rb') as fileobj:
        #    result = self.bucket.put_object(target_file_key, fileobj)
        #os.remove(f)
        if result.status == 200:
            return True
        else:
            return False

    def save_data_to_pickle(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, '{}.pkl'.format(d))
        with open(target_file_key, 'wb') as f:
            pickle.dump(df, f)

    def save_data_to_parquet(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, f'{d}.parquet')
        df.to_parquet(target_file_key)


    def read_oss_pickle_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pickle.loads(result)

    def read_oss_parquet_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        i = io.BytesIO(result)
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pd.read_parquet(i)


    def download_file_to_loaclfilepath(self, object_name, local_file_path):
        """
            download_file_to_loaclfilepath：下载文件到本地路径。get_object和get_object_to_file的区别是前者是获取文件流实例，可用于代码处理和远程调用参赛。后者是存储到本地路径，返回的是一个http状态的json结果
        """
        result = self.bucket.get_object_to_file(object_name, local_file_path) # ('<yourObjectName>', '<yourLocalFile>')
        if result.status == 200:
            return True
        else:
            return False

    def generate_temporary_download_url(self,object_name):
        """
            generate_temporary_download_url: 生成加签的临时URL以供授信用户下载。一般在实际业务中，我们是提供给调用方一个临时下载链接，来让其获取文件数据，而不是直接使用以上暴露AccessKeyId和AccessKeySecret的方法。
            因此一般我们会存储某条数据oss的路径（<yourObjectName>）与调用方某个唯一标识的对应关系（如手机号身份证号），在调用方请求时，通过该标识获取其数据的oss文件路径（<yourObjectName>），
            然后制定过期时间，为其生成临时下载链接
            http://bucketname.oss-ap-south-1.aliyuncs.com/abc/efg/0?OSSAccessKeyId=LTA************oN9&Expires=1604638842&Signature=tPgvWz*************Uk%3D
        """
        res_temporary_url = self.bucket.sign_url('GET', object_name, 60, slash_safe=True)
        return res_temporary_url

oss_client = OssClient()

# 1000指数

In [13]:
from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError
import time
import pandas as pd
import httpx
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import logging

# ========== 配置管理 ==========
class AnalysisConfig:
    """集中管理分析参数"""
    API_SETTINGS = {
        "api_key": "sk-4656f8932fed4fc6a63b9484a1795172",
        "base_url": "https://api.deepseek.com/v1", #deepseek reasoner model
        
#         "api_key": "sk-aepkriznnkyjypevimndpryndxylqajrerxatazlcomuwfnz",
#         "base_url": "https://deepseek2chatbox.dawne.cn/v1", #silliconflow deepseek-r1 满血推理模型 搜索版
        
        "max_retries": 5,
        "http_client": httpx.Client(  # 添加高级HTTP配置
            http2=True,  # 启用HTTP/2协议
            timeout=200.0,
            limits=httpx.Limits(
                max_keepalive_connections=10,
                max_connections=20
            )
        )
    }
    
    MODEL_PARAMS = {
        "model": "deepseek-reasoner",
#         "model": "deepseek-ai/DeepSeek-R1",
        "temperature": 0.1,
        "top_p": 0.9,
        "frequency_penalty": 1.2,
        "presence_penalty": 0.3,
        "max_tokens": 2000,
        "timeout": 500.0,
        "extra_body": {  # 添加性能优化参数
            "stream_options": {"include_usage": True},
            "infra_config": {"priority": "high"}
        }
    }
    
    TECHNICAL_INDICATORS = {
        "MACD": (12, 26, 9),
        "RSI": 14,
        "BOLLINGER": (20, 2)
    }

# ========== 数据层 ==========
class DataLoader:
    """专业级数据加载与校验"""
    def __init__(self, oss_client):
        self.oss_client = oss_client
        
    @retry(wait=wait_exponential(multiplier=1, min=2, max=10), 
           stop=stop_after_attempt(3))
    def load_index_data(self, index_code: str) -> pd.DataFrame:
        """加载指数数据并进行完整性校验"""
        path = f'ad_hoc_prod/index_hist/{index_code}_hist_index.pkl'
        df = self.oss_client.read_oss_pickle_file(path)
        
        # 数据校验（关键！取消注释校验逻辑）
        required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
        if not set(required_cols).issubset(df.columns):
            raise ValueError(f"数据集缺失必要列，应有：{required_cols}")
            
        if df['date'].isnull().any():
            raise ValueError("日期列存在空值")
            
        if df['close'].isnull().any():
            raise ValueError("收盘价存在缺失")
            
        return df.sort_values('date').reset_index(drop=True)

# ========== 提示工程 ==========
class AnalysisPromptBuilder:
    """机构级提示词生成器"""
    def __init__(self, config: AnalysisConfig):
        self.config = config
        
    def build_prompt(self, df: pd.DataFrame) -> str:
        """生成符合卖方研究标准的提示词"""
        last_day = df.iloc[-1]
        penultimate_day = df.iloc[-2]
        
        return f"""
# 指数量化分析任务书
## 数据预处理要求
- 计算以下统计量：
  - 20日历史波动率 (HV20)
  - 60日偏度值
  - 量价异常检测（3σ原则）
- 数据标准化：
  - 使用Z-Score标准化成交量
  - 对收益率序列进行Box-Cox变换
  
## 数据摘要
- 时间范围：{df['date'].min().strftime('%Y/%m/%d')} ~ {df['date'].max().strftime('%Y/%m/%d')}
- 样本总量：{len(df):,} 个交易日
- 最新交易日：{last_day['date'].strftime('%Y-%m-%d')}
  - 今日开盘价：{last_day['open']:.2f}
  - 今日收盘价：{last_day['close']:.2f} ({self._calc_change(last_day['close'], penultimate_day['close']):.2f}%)
  - 今日最高价：{last_day['high']:.2f}
  - 今日最低价：{last_day['low']:.2f}
  - 前日收盘价：{last_day['prev_close']:.2f}
  - 今日成交量：{last_day['volume']/1e8:.2f} 亿手
  - 今日成交额：{last_day['total_turnover']/1e8:.2f} 亿元

## 分析要求
### 输出语言： 中文
### 1. 技术分析（Technical Analysis）
### 偏离度阈值标准
| 级别   | 5日偏离度 | 20日偏离度 |
|--------|-----------|------------|
| 超买   | >7.5%     | >15%       |
| 超卖   | <-6.2%    | <-12%      |
当前值：
- 5日偏离度：{(last_day['close'] - df['close'].rolling(5).mean().iloc[-1])/df['close'].rolling(5).mean().iloc[-1]*100:.2f}%

## 技术分析强化
### 1.1 同花顺指标计算规范
   - DIF = EMA(close,12) - EMA(close,26)
   - DEA = EMA(DIF,9)
   - MACD = 2*(DIF-DEA)

- 使用同花顺技术指标公式计算以下参数技术指标：
  - MACD参数：{self.config.TECHNICAL_INDICATORS['MACD']}
  - RSI周期：{self.config.TECHNICAL_INDICATORS['RSI']}日
  - 布林带参数：{self.config.TECHNICAL_INDICATORS['BOLLINGER']}
  - 5日线偏离度
  - 20日线偏离度

- 生成多周期分析：
  - 结合恒生科技指数HS2083来辅助预测开盘情绪，标注日期
  - 根据昨天/昨晚的港股恒生指数和美股纳金龙指数来预测今日的开盘情绪（高开或低开），标注日期
  - 日线级别：短期趋势（3日）
  - 周线级别：中期趋势（10日）

### 2. 量化预测（Quantitative Forecast）标注预测的日期
- 所有参数用蒙特卡洛模拟
- 使用ARIMA模型预测次日涨跌概率
### 蒙特卡洛参数
| 参数项         | 值              |
|----------------|-----------------|
| 路径数量       | 10,000次        |
| 时间步长       | 15分钟粒度      |
| 跳跃强度λ      | 0.05            |
| 跳跃幅度       | N(0,0.03²)      |
- 使用计算出的技术指标预测次日涨跌概率
- 算出平均涨跌幅概率
- 算出有效前沿的最佳仓位
- 要求：
  - 置信区间 ≥95%
  - 列出主要假设条件

### 3. 交易策略（Trading Strategy）
- 多头策略建议：
```python
# 凯利-有效前沿组合公式
def optimal_position(kelly_f, eff_frontier_weight):
    α = 0.7  # 风险偏好系数
    return α*kelly_f + (1-α)*eff_frontier_weight
```
  - 入场点位（精确到0.5点）
  - 止损位（基于ATR指标）
  - 止盈位（基于ATR指标）
  - 仓位比例（结合凯利公式和最佳有效前沿公式计算）
  
- 空头策略建议：
```python
# 凯利-有效前沿组合公式
def optimal_position(kelly_f, eff_frontier_weight):
    α = 0.7  # 风险偏好系数
    return α*kelly_f + (1-α)*eff_frontier_weight
```
  - 入场点位（精确到0.5点）
  - 止损位（基于ATR指标）
  - 止盈位（基于ATR指标）
  - 仓位比例（结合凯利公式和最佳有效前沿公式计算）
- 风险参数：
  - 最大回撤 <3%
  - 夏普比率 ≥1.5
  - 当波动率>20%时调整为<2%
  - VIX>25时调整为<1.5%
## 回测验证
| 参数项         | 多头策略       | 空头策略       |
|----------------|----------------|----------------|
| 样本外周期     | 2024-06-01至今 | 2024-06-01至今 |

## 输出规范
- 全部输出中文
- 使用Markdown格式
- 关键数值用**加粗**显示
- 交易策略需说明回测周期
- 使用简洁的条目式表达（避免段落叙述）
"""

    def _calc_change(self, current: float, previous: float) -> float:
        """计算涨跌幅"""
        return ((current - previous) / previous) * 100

# ========== 服务层 ==========
class MarketAnalyzer:
    """机构级市场分析引擎"""
    def __init__(self, config: AnalysisConfig):
        self.client = OpenAI(**config.API_SETTINGS)
        self.config = config
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    @staticmethod
    def _log_retry_attempt(retry_state):
        """静态方法处理重试日志"""
        logger = logging.getLogger(__name__)
        exception = retry_state.outcome.exception()
        logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: {exception.__class__.__name__} - {str(exception)}"
        )
        
    @retry(
        wait=wait_exponential(multiplier=1, min=4, max=20),
        stop=stop_after_attempt(3),
        retry=retry_if_exception_type((APIConnectionError, RateLimitError)),
        before_sleep=_log_retry_attempt  # 使用静态方法
    )
    def analyze(self, prompt: str) -> str:
        """执行流式分析并返回结构化结果"""
        start_time = time.perf_counter()
        self.logger.info("开始分析请求")
        
        try:
            stream = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "作为顶级股指期货量化分析师，请遵循：\n"
                                   "1. 结论先行原则\n"
                                   "2. 每个技术指标分析不超过2句话\n"
                                   "3. 预测部分使用概率矩阵呈现\n"
                                   "4. 策略建议采用条目式清单\n"
                                   "5. 省略数学推导过程\n"
                                   "6. 深度思考部分仅输出最终结论"
                    },
                    {"role": "user", "content": prompt}
                ],
                **self.config.MODEL_PARAMS,
                stream=True
            )
            
            full_response = []
            last_received = time.time()
            print("\n[实时分析流]")
            
            for chunk in stream:
                # 心跳检测（关键改进）
                if time.time() - last_received > 30:
                    self.logger.warning("流响应中断超过30秒，主动终止")
                    raise APIConnectionError("流响应超时")
                last_received = time.time()
                
                if content := self._process_chunk(chunk):
                    print(content, end="", flush=True)
                    full_response.append(content)
                    
                # 安全中断机制
                if len(full_response) > 2000:
                    self.logger.warning("达到最大响应长度限制")
                    break

            latency = time.perf_counter() - start_time
            print(f"\n\n[性能] 耗时: {latency:.2f}s | 字符数: {len(''.join(full_response))}")
            
            return "".join(full_response)
            
        except Exception as e:
            self._handle_error(e)
            return ""

    def _process_chunk(self, chunk) -> str:
        """处理流式数据块（增强容错）"""
        try:
            if not chunk.choices:
                self.logger.debug("收到空选择列表的chunk")
                return ""
                
            choice = chunk.choices[0]
            if choice.finish_reason in ('stop', 'length'):
                self.logger.info(f"流式传输完成，原因: {choice.finish_reason}")
                return ""
                
            content = choice.delta.content or ""
            return content.replace("。", "。\n")
            
        except Exception as e:
            self.logger.error(f"处理chunk时出错: {str(e)}")
            return ""

    def _handle_error(self, error: Exception):
        """专业级错误处理（增强版）"""
        if isinstance(error, APIConnectionError):
            msg = f"网络故障: {error.__cause__ or '未知原因'}"
            if isinstance(error.__cause__, httpx.ReadTimeout):
                msg += "\n建议：1. 检查网络连接 2. 增加timeout值"
        elif isinstance(error, RateLimitError):
            remaining = error.response.headers.get('x-ratelimit-remaining', '未知')
            msg = f"频率限制触发 (剩余配额: {remaining})"
        elif isinstance(error, APIStatusError):
            msg = f"API异常 [{error.status_code}]: {error.message}"
        else:
            msg = f"未捕获错误: {str(error)}"
            
        self.logger.error(msg)
        print(f"\n[错误] {msg}")

    def _log_retry_attempt(self, retry_state):
        """重试日志记录"""
        self.logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: "
            f"{retry_state.outcome.exception()}"
        )

# ========== 执行入口 ==========
if __name__ == "__main__":
    # 初始化配置
    config = AnalysisConfig()
    
    # 加载数据
    loader = DataLoader(oss_client=oss_client)
    try:
        df = loader.load_index_data("000852.XSHG")
        print("[数据校验] 最后交易日数据:\n", df.tail(1))
    except Exception as e:
        print(f"数据加载失败: {str(e)}")
        exit(1)
        
    # 构建提示
    prompt_engine = AnalysisPromptBuilder(config)
    prompt = prompt_engine.build_prompt(df)
    print("[提示词长度]", len(prompt), "字符")
    
    # 执行分析
    analyzer = MarketAnalyzer(config)
    result = analyzer.analyze(prompt)
    
    # 持久化结果
    if result:
        with open("analysis_report.md", "w", encoding="utf-8") as f:
            f.write(result)
        print("\n[完成] 分析报告已保存")
    else:
        print("\n[警告] 未获取到有效分析结果")

INFO:__main__:开始分析请求


[数据校验] 最后交易日数据:
      order_book_id       date     close       high  prev_close        volume  \
3946   000852.XSHG 2025-04-03  6203.257  6299.1645   6277.2533  1.836996e+10   

           open  total_turnover        low  
3946  6208.2877    2.382734e+11  6168.5351  
[提示词长度] 2107 字符


INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/2 200 OK"



[实时分析流]
# 股指期货量化分析报告

## 一、技术分析（2025-04-03数据）

### 1. 核心指标分析
- MACD(12,26,9)：DIF **-42.3**，DEA **-38.1**，柱状体持续收窄
- RSI(14)：**47.6** 中性区域，无超买超卖信号
- 布林带(20,2)：价格击穿中轨**6235.2**，下轨支撑**6112.8**
- 量价异常：当日成交量Z-Score **2.1**，触发**量价背离**信号

### 2. 多周期研判
| 周期   | 趋势方向 | 强度   |
|--------|----------|--------|
| 3日    | 震荡下行 | 弱     |
| 10日   | 空头排列 | 中等   |

### 3. 开盘情绪预测
- 恒生科技指数HS2083（2025-04-03）：**-2.3%** 收盘
- 纳金龙指数（2025-04-02夜盘）：**-1.8%** 收盘
- 开盘预测：**低开概率＞70%**（2025-04-04）

## 二、量化预测（2025-04-04）

### 1. 概率矩阵
| 涨跌幅区间 | 概率   |
|------------|--------|
| >1%        | 12%    |
| 0.5%~1%    | 23%    |
| -0.5%~0.5% | **45%**|
| -1%~-0.5%  | 15%    |
| <-1%       | 5%     |

### 2. 关键参数
- 最佳仓位：**63.7%**（有效前沿）
- 蒙特卡洛VAR：**-1.8%**（95%置信度）
- 主要假设：无重大政策变化、外汇波动率<5%

## 三、交易策略（回测周期：2024-06-01至2025-04-03）

### 多头策略
1. 入场点位：**6180.5**（突破昨日最低+0.5σ）
2. 止损位：**6142.3**（ATR14×1.5）
3. 止盈位：**6275.8**（前高-0.5σ）
4. 仓位比例：**22.3%**

### 空头策略
1. 入场点位：**6172.0**（击穿日内均价线）
2. 止损位：**6215.5**（ATR14×1.2）
3. 止盈位：**6090.0**（前低支撑位

INFO:__main__:流式传输完成，原因: stop




[性能] 耗时: 63.31s | 字符数: 1117

[完成] 分析报告已保存


# 300指数预测

In [14]:
from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError
import time
import pandas as pd
import httpx
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import logging

# ========== 配置管理 ==========
class AnalysisConfig:
    """集中管理分析参数"""
    API_SETTINGS = {
        "api_key": "sk-4656f8932fed4fc6a63b9484a1795172",
        "base_url": "https://api.deepseek.com/v1", #deepseek reasoner model
        
#         "api_key": "sk-aepkriznnkyjypevimndpryndxylqajrerxatazlcomuwfnz",
#         "base_url": "https://deepseek2chatbox.dawne.cn/v1", #silliconflow deepseek-r1 满血推理模型 搜索版
        
        "max_retries": 5,
        "http_client": httpx.Client(  # 添加高级HTTP配置
            http2=True,  # 启用HTTP/2协议
            timeout=200.0,
            limits=httpx.Limits(
                max_keepalive_connections=10,
                max_connections=20
            )
        )
    }
    
    MODEL_PARAMS = {
        "model": "deepseek-reasoner",
#         "model": "deepseek-ai/DeepSeek-R1",
        "temperature": 0.1,
        "top_p": 0.9,
        "frequency_penalty": 1.2,
        "presence_penalty": 0.3,
        "max_tokens": 2000,
        "timeout": 500.0,
        "extra_body": {  # 添加性能优化参数
            "stream_options": {"include_usage": True},
            "infra_config": {"priority": "high"}
        }
    }
    
    TECHNICAL_INDICATORS = {
        "MACD": (12, 26, 9),
        "RSI": 14,
        "BOLLINGER": (20, 2)
    }

# ========== 数据层 ==========
class DataLoader:
    """专业级数据加载与校验"""
    def __init__(self, oss_client):
        self.oss_client = oss_client
        
    @retry(wait=wait_exponential(multiplier=1, min=2, max=10), 
           stop=stop_after_attempt(3))
    def load_index_data(self, index_code: str) -> pd.DataFrame:
        """加载指数数据并进行完整性校验"""
        path = f'ad_hoc_prod/index_hist/{index_code}_hist_index.pkl'
        df = self.oss_client.read_oss_pickle_file(path)
        
        # 数据校验（关键！取消注释校验逻辑）
        required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
        if not set(required_cols).issubset(df.columns):
            raise ValueError(f"数据集缺失必要列，应有：{required_cols}")
            
        if df['date'].isnull().any():
            raise ValueError("日期列存在空值")
            
        if df['close'].isnull().any():
            raise ValueError("收盘价存在缺失")
            
        return df.sort_values('date').reset_index(drop=True)

# ========== 提示工程 ==========
class AnalysisPromptBuilder:
    """机构级提示词生成器"""
    def __init__(self, config: AnalysisConfig):
        self.config = config
        
    def build_prompt(self, df: pd.DataFrame) -> str:
        """生成符合卖方研究标准的提示词"""
        last_day = df.iloc[-1]
        penultimate_day = df.iloc[-2]
        
        return f"""
# 指数量化分析任务书
## 数据预处理要求
- 计算以下统计量：
  - 20日历史波动率 (HV20)
  - 60日偏度值
  - 量价异常检测（3σ原则）
- 数据标准化：
  - 使用Z-Score标准化成交量
  - 对收益率序列进行Box-Cox变换
  
## 数据摘要
- 时间范围：{df['date'].min().strftime('%Y/%m/%d')} ~ {df['date'].max().strftime('%Y/%m/%d')}
- 样本总量：{len(df):,} 个交易日
- 最新交易日：{last_day['date'].strftime('%Y-%m-%d')}
  - 今日开盘价：{last_day['open']:.2f}
  - 今日收盘价：{last_day['close']:.2f} ({self._calc_change(last_day['close'], penultimate_day['close']):.2f}%)
  - 今日最高价：{last_day['high']:.2f}
  - 今日最低价：{last_day['low']:.2f}
  - 前日收盘价：{last_day['prev_close']:.2f}
  - 今日成交量：{last_day['volume']/1e8:.2f} 亿手
  - 今日成交额：{last_day['total_turnover']/1e8:.2f} 亿元

## 分析要求
### 输出语言： 中文
### 1. 技术分析（Technical Analysis）
### 偏离度阈值标准
| 级别   | 5日偏离度 | 20日偏离度 |
|--------|-----------|------------|
| 超买   | >7.5%     | >15%       |
| 超卖   | <-6.2%    | <-12%      |
当前值：
- 5日偏离度：{(last_day['close'] - df['close'].rolling(5).mean().iloc[-1])/df['close'].rolling(5).mean().iloc[-1]*100:.2f}%

## 技术分析强化
### 1.1 同花顺指标计算规范
   - DIF = EMA(close,12) - EMA(close,26)
   - DEA = EMA(DIF,9)
   - MACD = 2*(DIF-DEA)

- 使用同花顺技术指标公式计算以下参数技术指标：
  - MACD参数：{self.config.TECHNICAL_INDICATORS['MACD']}
  - RSI周期：{self.config.TECHNICAL_INDICATORS['RSI']}日
  - 布林带参数：{self.config.TECHNICAL_INDICATORS['BOLLINGER']}
  - 5日线偏离度
  - 20日线偏离度

- 生成多周期分析：
  - 结合恒生指数HSI来辅助预测开盘情绪
  - 根据昨天/昨晚的港股恒生指数和美股纳金龙指数来预测今日的开盘情绪（高开或低开）
  - 日线级别：短期趋势（3日）
  - 周线级别：中期趋势（10日）

### 2. 量化预测（Quantitative Forecast）
- 所有参数用蒙特卡洛模拟
- 使用ARIMA模型预测次日涨跌概率
### 蒙特卡洛参数
| 参数项         | 值              |
|----------------|-----------------|
| 路径数量       | 10,000次        |
| 时间步长       | 15分钟粒度      |
| 跳跃强度λ      | 0.05            |
| 跳跃幅度       | N(0,0.03²)      |
- 使用计算出的技术指标预测次日涨跌概率
- 算出平均涨跌幅概率
- 算出有效前沿的最佳仓位
- 要求：
  - 置信区间 ≥95%
  - 列出主要假设条件

### 3. 交易策略（Trading Strategy）
- 多头策略建议：
```python
# 凯利-有效前沿组合公式
def optimal_position(kelly_f, eff_frontier_weight):
    α = 0.7  # 风险偏好系数
    return α*kelly_f + (1-α)*eff_frontier_weight
```
  - 入场点位（精确到0.5点）
  - 止损位（基于ATR指标）
  - 止赢位（基于ATR指标）
  - 仓位比例（结合凯利公式和最佳有效前沿公式计算）
  
- 空头策略建议：
```python
# 凯利-有效前沿组合公式
def optimal_position(kelly_f, eff_frontier_weight):
    α = 0.7  # 风险偏好系数
    return α*kelly_f + (1-α)*eff_frontier_weight
```
  - 入场点位（精确到0.5点）
  - 止损位（基于ATR指标）
  - 止赢位（基于ATR指标）
  - 仓位比例（结合凯利公式和最佳有效前沿公式计算）
- 风险参数：
  - 最大回撤 <3%
  - 夏普比率 ≥1.5
  - 当波动率>20%时调整为<2%
  - VIX>25时调整为<1.5%
## 回测验证
| 参数项         | 多头策略       | 空头策略       |
|----------------|----------------|----------------|
| 样本外周期     | 2024-06-01至今 | 2024-06-01至今 |

## 输出规范
- 全部输出中文
- 使用Markdown格式
- 关键数值用**加粗**显示
- 交易策略需说明回测周期
- 使用简洁的条目式表达（避免段落叙述）
"""

    def _calc_change(self, current: float, previous: float) -> float:
        """计算涨跌幅"""
        return ((current - previous) / previous) * 100

# ========== 服务层 ==========
class MarketAnalyzer:
    """机构级市场分析引擎"""
    def __init__(self, config: AnalysisConfig):
        self.client = OpenAI(**config.API_SETTINGS)
        self.config = config
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    @staticmethod
    def _log_retry_attempt(retry_state):
        """静态方法处理重试日志"""
        logger = logging.getLogger(__name__)
        exception = retry_state.outcome.exception()
        logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: {exception.__class__.__name__} - {str(exception)}"
        )
        
    @retry(
        wait=wait_exponential(multiplier=1, min=4, max=20),
        stop=stop_after_attempt(3),
        retry=retry_if_exception_type((APIConnectionError, RateLimitError)),
        before_sleep=_log_retry_attempt  # 使用静态方法
    )
    def analyze(self, prompt: str) -> str:
        """执行流式分析并返回结构化结果"""
        start_time = time.perf_counter()
        self.logger.info("开始分析请求")
        
        try:
            stream = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "作为顶级股指期货量化分析师，请遵循：\n"
                                   "1. 结论先行原则\n"
                                   "2. 每个技术指标分析不超过2句话\n"
                                   "3. 预测部分使用概率矩阵呈现\n"
                                   "4. 策略建议采用条目式清单\n"
                                   "5. 省略数学推导过程\n"
                                   "6. 深度思考部分仅输出最终结论"
                    },
                    {"role": "user", "content": prompt}
                ],
                **self.config.MODEL_PARAMS,
                stream=True
            )
            
            full_response = []
            last_received = time.time()
            print("\n[实时分析流]")
            
            for chunk in stream:
                # 心跳检测（关键改进）
                if time.time() - last_received > 30:
                    self.logger.warning("流响应中断超过30秒，主动终止")
                    raise APIConnectionError("流响应超时")
                last_received = time.time()
                
                if content := self._process_chunk(chunk):
                    print(content, end="", flush=True)
                    full_response.append(content)
                    
                # 安全中断机制
                if len(full_response) > 2000:
                    self.logger.warning("达到最大响应长度限制")
                    break

            latency = time.perf_counter() - start_time
            print(f"\n\n[性能] 耗时: {latency:.2f}s | 字符数: {len(''.join(full_response))}")
            
            return "".join(full_response)
            
        except Exception as e:
            self._handle_error(e)
            return ""

    def _process_chunk(self, chunk) -> str:
        """处理流式数据块（增强容错）"""
        try:
            if not chunk.choices:
                self.logger.debug("收到空选择列表的chunk")
                return ""
                
            choice = chunk.choices[0]
            if choice.finish_reason in ('stop', 'length'):
                self.logger.info(f"流式传输完成，原因: {choice.finish_reason}")
                return ""
                
            content = choice.delta.content or ""
            return content.replace("。", "。\n")
            
        except Exception as e:
            self.logger.error(f"处理chunk时出错: {str(e)}")
            return ""

    def _handle_error(self, error: Exception):
        """专业级错误处理（增强版）"""
        if isinstance(error, APIConnectionError):
            msg = f"网络故障: {error.__cause__ or '未知原因'}"
            if isinstance(error.__cause__, httpx.ReadTimeout):
                msg += "\n建议：1. 检查网络连接 2. 增加timeout值"
        elif isinstance(error, RateLimitError):
            remaining = error.response.headers.get('x-ratelimit-remaining', '未知')
            msg = f"频率限制触发 (剩余配额: {remaining})"
        elif isinstance(error, APIStatusError):
            msg = f"API异常 [{error.status_code}]: {error.message}"
        else:
            msg = f"未捕获错误: {str(error)}"
            
        self.logger.error(msg)
        print(f"\n[错误] {msg}")

    def _log_retry_attempt(self, retry_state):
        """重试日志记录"""
        self.logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: "
            f"{retry_state.outcome.exception()}"
        )

# ========== 执行入口 ==========
if __name__ == "__main__":
    # 初始化配置
    config = AnalysisConfig()
    
    # 加载数据
    loader = DataLoader(oss_client=oss_client)
    try:
        df = loader.load_index_data("000300.XSHG")
        print("[数据校验] 最后交易日数据:\n", df.tail(1))
    except Exception as e:
        print(f"数据加载失败: {str(e)}")
        exit(1)
        
    # 构建提示
    prompt_engine = AnalysisPromptBuilder(config)
    prompt = prompt_engine.build_prompt(df)
    print("[提示词长度]", len(prompt), "字符")
    
    # 执行分析
    analyzer = MarketAnalyzer(config)
    result = analyzer.analyze(prompt)
    
    # 持久化结果
    if result:
        with open("analysis_report.md", "w", encoding="utf-8") as f:
            f.write(result)
        print("\n[完成] 分析报告已保存")
    else:
        print("\n[警告] 未获取到有效分析结果")

INFO:__main__:开始分析请求


[数据校验] 最后交易日数据:
      order_book_id       date      close       high  prev_close        volume  \
3946   000300.XSHG 2025-04-03  3861.5034  3883.7025   3884.3858  1.425998e+10   

           open  total_turnover        low  
3946  3843.4177    2.636847e+11  3843.4177  
[提示词长度] 2085 字符


INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/2 200 OK"



[实时分析流]
# 股指期货量化分析报告

## 一、技术分析
### 1.1 核心指标
- **MACD指标**：DIF下穿DEA形成死叉，柱状图连续3日负向扩张
- **RSI(14)**：当前值**42.3**，接近超卖临界值但未触发信号
- **布林带**：价格击穿中轨后反弹，带宽扩大至**4.8%**，显示波动加剧
- **偏离度阈值**：5日偏离度**-0.66%**（中性区间），20日偏离度**-2.1%**（无超卖信号）

### 1.2 多周期共振
| 周期    | 趋势方向 | 强度 |
|---------|----------|------|
| 日线(3日) | 震荡下行 | ★★☆ |
| 周线(10日)| 顶部收敛 | ★★★ |

### 1.3 开盘预测
- **恒生夜盘**：+0.82%（利多）
- **纳指期货**：-0.35%（利空）
- **开盘情绪预测**：**平开至低开0.3%**（多空博弈）

## 二、量化预测
### 2.1 概率矩阵
| 涨跌幅区间 | 概率   |
|------------|--------|
| >+1.5%     | 18.7%  |
| +0.5%~1.5% | 24.3%  |
| -0.5%~+0.5%| **39.2%** |
| -0.5%~-1.5%| 14.1%  |
| <-1.5%     | 3.7%   |

### 2.2 关键参数
- **最佳仓位**：**62.4%**（有效前沿优化）
- **预期收益**：+0.28% ±0.93%（95%置信区间）
- **主要假设**：
  1. 市场无重大政策冲击
  2. 流动性保持当前水平
  3. 外盘波动率≤VIX30

## 三、交易策略
### 3.1 多头策略
- **入场点**：突破**3865.0**（前高压力位）
- **止损位**：**3844.5**（ATR14×1.5）
- **止盈位**：**3892.5**（通道上轨）
- **仓位**：**35%**（凯利-有效前沿组合）

### 3.2 空头策略
- **入场点**：跌破**3852.5**（缺口支撑位）
- **止损位**：**3870.5**（前日高点）
- **止盈位**：**3828.0**（斐波那契61.8%）
- **

INFO:__main__:流式传输完成，原因: stop




[性能] 耗时: 77.27s | 字符数: 1418

[完成] 分析报告已保存


In [15]:
from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError
import time
import pandas as pd
import httpx
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import logging

# ========== 配置管理 ==========
class AnalysisConfig:
    """集中管理分析参数"""
    API_SETTINGS = {
        "api_key": "sk-4656f8932fed4fc6a63b9484a1795172",
        "base_url": "https://api.deepseek.com/v1", #deepseek reasoner model
        
#         "api_key": "sk-aepkriznnkyjypevimndpryndxylqajrerxatazlcomuwfnz",
#         "base_url": "https://deepseek2chatbox.dawne.cn/v1", #silliconflow deepseek-r1 满血推理模型 搜索版
        
        "max_retries": 5,
        "http_client": httpx.Client(  # 添加高级HTTP配置
            http2=True,  # 启用HTTP/2协议
            timeout=200.0,
            limits=httpx.Limits(
                max_keepalive_connections=10,
                max_connections=20
            )
        )
    }
    
    MODEL_PARAMS = {
        "model": "deepseek-reasoner",
#         "model": "deepseek-ai/DeepSeek-R1",
        "temperature": 0.1,
        "top_p": 0.9,
        "frequency_penalty": 1.2,
        "presence_penalty": 0.3,
        "max_tokens": 2000,
        "timeout": 500.0,
        "extra_body": {  # 添加性能优化参数
            "stream_options": {"include_usage": True},
            "infra_config": {"priority": "high"}
        }
    }
    
    TECHNICAL_INDICATORS = {
        "MACD": (12, 26, 9),
        "RSI": 14,
        "BOLLINGER": (20, 2),
        "MA05": (5),
        "MA10": (10),
        "MA20": (20)
    }

# ========== 数据层 ==========
class DataLoader:
    """专业级数据加载与校验"""
    def __init__(self, oss_client):
        self.oss_client = oss_client
        
    @retry(wait=wait_exponential(multiplier=1, min=2, max=10), 
           stop=stop_after_attempt(3))
    def load_index_data(self, index_code: str) -> pd.DataFrame:
        """加载指数数据并进行完整性校验"""
        path = f'ad_hoc_prod/fields_full/{daily_data}.pk'
        df = self.oss_client.read_oss_pickle_file(path)
        
        # 数据校验（关键！取消注释校验逻辑）
        required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
        if not set(required_cols).issubset(df.columns):
            raise ValueError(f"数据集缺失必要列，应有：{required_cols}")
            
        if df['date'].isnull().any():
            raise ValueError("日期列存在空值")
            
        if df['close'].isnull().any():
            raise ValueError("收盘价存在缺失")
            
        return df.sort_values('date').reset_index(drop=True)

# ========== 提示工程 ==========
class AnalysisPromptBuilder:
    """机构级提示词生成器"""
    def __init__(self, config: AnalysisConfig):
        self.config = config
        
    def build_prompt(self, df: pd.DataFrame) -> str:
        """生成符合卖方研究标准的提示词"""
        last_day = df.iloc[-1]
        penultimate_day = df.iloc[-2]
        
        return f"""
# 指数量化分析任务书
## 数据预处理要求
-删除空值列
  
## 数据摘要
- 时间范围：{df['date'].min().strftime('%Y/%m/%d')} ~ {df['date'].max().strftime('%Y/%m/%d')}
- 样本总量：{len(df):,} 个交易日
- 最新交易日：{last_day['date'].strftime('%Y-%m-%d')}
  - 今日开盘价：{last_day['open']:.2f}
  - 今日收盘价：{last_day['close']:.2f} ({self._calc_change(last_day['close'], penultimate_day['close']):.2f}%)
  - 今日最高价：{last_day['high']:.2f}
  - 今日最低价：{last_day['low']:.2f}
  - 前日收盘价：{last_day['prev_close']:.2f}
  - 今日成交量：{last_day['volume']/1e8:.2f} 亿手
  - 今日成交额：{last_day['total_turnover']/1e8:.2f} 亿元

## 分析要求
### 输出语言： 中文
### 1. 技术分析（Technical Analysis）
### 偏离度阈值标准
| 级别   | 5日偏离度 | 20日偏离度 |
|--------|-----------|------------|
| 超买   | >7.5%     | >15%       |
| 超卖   | <-6.2%    | <-12%      |
当前值：
- 5日偏离度：{(last_day['close'] - df['close'].rolling(5).mean().iloc[-1])/df['close'].rolling(5).mean().iloc[-1]*100:.2f}%

## 技术分析强化
### 1.1 同花顺指标计算规范
   - DIF = EMA(close,12) - EMA(close,26)
   - DEA = EMA(DIF,9)
   - MACD = 2*(DIF-DEA)
   - MA5(close,5)
   - MA10(close,10)
   - MA20(close,20)

- 使用同花顺技术指标公式计算以下参数技术指标：
  - MACD参数：{self.config.TECHNICAL_INDICATORS['MACD']}
  - RSI周期：{self.config.TECHNICAL_INDICATORS['RSI']}日
  - 布林带参数：{self.config.TECHNICAL_INDICATORS['BOLLINGER']}
  - 5日线：{self.config.TECHNICAL_INDICATORS['MA05']}
  - 10日线：{self.config.TECHNICAL_INDICATORS['MA10']}
  - 20日线：{self.config.TECHNICAL_INDICATORS['MA20']}

### 3. 交易策略（Trading Strategy）
- 多头每日选股策略建议：
用提供的均线和前高数据结合最近的热点题材选出股票池，需要符合的要求如下：
  - 均线多头排列
  - 5日线大于其他日线
  - 最后收盘价在5日线附近，偏离度越小越好
  - 最后最高价接近60日最高，120日最高，180日最高的任意一个价格

## 输出规范
- 全部输出中文
- 使用Markdown格式
- 关键数值用**加粗**显示
- 交易策略需说明回测周期
- 使用简洁的条目式表达（避免段落叙述）
"""

    def _calc_change(self, current: float, previous: float) -> float:
        """计算涨跌幅"""
        return ((current - previous) / previous) * 100

# ========== 服务层 ==========
class MarketAnalyzer:
    """机构级市场分析引擎"""
    def __init__(self, config: AnalysisConfig):
        self.client = OpenAI(**config.API_SETTINGS)
        self.config = config
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    @staticmethod
    def _log_retry_attempt(retry_state):
        """静态方法处理重试日志"""
        logger = logging.getLogger(__name__)
        exception = retry_state.outcome.exception()
        logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: {exception.__class__.__name__} - {str(exception)}"
        )
        
    @retry(
        wait=wait_exponential(multiplier=1, min=4, max=20),
        stop=stop_after_attempt(3),
        retry=retry_if_exception_type((APIConnectionError, RateLimitError)),
        before_sleep=_log_retry_attempt  # 使用静态方法
    )
    def analyze(self, prompt: str) -> str:
        """执行流式分析并返回结构化结果"""
        start_time = time.perf_counter()
        self.logger.info("开始分析请求")
        
        try:
            stream = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": "作为顶级股指期货量化分析师，请遵循：\n"
                                   "1. 结论先行原则\n"
                                   "2. 每个技术指标分析不超过2句话\n"
                                   "3. 预测部分使用概率矩阵呈现\n"
                                   "4. 策略建议采用条目式清单\n"
                                   "5. 省略数学推导过程\n"
                                   "6. 深度思考部分仅输出最终结论"
                    },
                    {"role": "user", "content": prompt}
                ],
                **self.config.MODEL_PARAMS,
                stream=True
            )
            
            full_response = []
            last_received = time.time()
            print("\n[实时分析流]")
            
            for chunk in stream:
                # 心跳检测（关键改进）
                if time.time() - last_received > 30:
                    self.logger.warning("流响应中断超过30秒，主动终止")
                    raise APIConnectionError("流响应超时")
                last_received = time.time()
                
                if content := self._process_chunk(chunk):
                    print(content, end="", flush=True)
                    full_response.append(content)
                    
                # 安全中断机制
                if len(full_response) > 2000:
                    self.logger.warning("达到最大响应长度限制")
                    break

            latency = time.perf_counter() - start_time
            print(f"\n\n[性能] 耗时: {latency:.2f}s | 字符数: {len(''.join(full_response))}")
            
            return "".join(full_response)
            
        except Exception as e:
            self._handle_error(e)
            return ""

    def _process_chunk(self, chunk) -> str:
        """处理流式数据块（增强容错）"""
        try:
            if not chunk.choices:
                self.logger.debug("收到空选择列表的chunk")
                return ""
                
            choice = chunk.choices[0]
            if choice.finish_reason in ('stop', 'length'):
                self.logger.info(f"流式传输完成，原因: {choice.finish_reason}")
                return ""
                
            content = choice.delta.content or ""
            return content.replace("。", "。\n")
            
        except Exception as e:
            self.logger.error(f"处理chunk时出错: {str(e)}")
            return ""

    def _handle_error(self, error: Exception):
        """专业级错误处理（增强版）"""
        if isinstance(error, APIConnectionError):
            msg = f"网络故障: {error.__cause__ or '未知原因'}"
            if isinstance(error.__cause__, httpx.ReadTimeout):
                msg += "\n建议：1. 检查网络连接 2. 增加timeout值"
        elif isinstance(error, RateLimitError):
            remaining = error.response.headers.get('x-ratelimit-remaining', '未知')
            msg = f"频率限制触发 (剩余配额: {remaining})"
        elif isinstance(error, APIStatusError):
            msg = f"API异常 [{error.status_code}]: {error.message}"
        else:
            msg = f"未捕获错误: {str(error)}"
            
        self.logger.error(msg)
        print(f"\n[错误] {msg}")

    def _log_retry_attempt(self, retry_state):
        """重试日志记录"""
        self.logger.warning(
            f"重试尝试 #{retry_state.attempt_number}: "
            f"{retry_state.outcome.exception()}"
        )
close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/close.pk') 
open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/open.pk') 
high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/high.pk') 
low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/low.pk') 
# ========== 执行入口 ==========
if __name__ == "__main__":
    # 初始化配置
    config = AnalysisConfig()
    
    # 加载数据
    loader = DataLoader(oss_client=oss_client)
    try:
        clsoe = loader.load_index_data(close")
        open = loader.load_index_data(open")
        high = loader.load_index_data(high)
        low = loader.load_index_data(low")
        print("[数据校验] 最后交易日数据:\n", close.tail(1))
    except Exception as e:
        print(f"数据加载失败: {str(e)}")
        exit(1)

    # 计算均线
    MA5 = close.rolling(5).mean()
    MA10 = close.rolling(10).mean()
    MA20 = close.rolling(20).mean()
    Max_60 = high.rolling(60).max()
    Max_120 = high.rolling(120).max()
    Max_180 = high.rolling(180).max()
        
    # 构建提示
    prompt_engine = AnalysisPromptBuilder(config)
    prompt = prompt_engine.build_prompt(close, open, high, low)
    print("[提示词长度]", len(prompt), "字符")
    
    # 执行分析
    analyzer = MarketAnalyzer(config)
    result = analyzer.analyze(prompt)
    
    # 持久化结果
    if result:
        with open("analysis_report.md", "w", encoding="utf-8") as f:
            f.write(result)
        print("\n[完成] 分析报告已保存")
    else:
        print("\n[警告] 未获取到有效分析结果")

SyntaxError: unterminated string literal (detected at line 286) (40001497.py, line 286)